In [1]:

#load clean data
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np
import re
import os

root_dir = os.getcwd()

In [2]:
#backout one level to get to the data
os.chdir('..')
data_dir = os.getcwd()

data = pd.read_csv('data/processed/df_xPassRush.csv')
games_df = pd.read_csv('data/raw/games.csv')

#keep only relevant columns from games_df
games_df = games_df[['gameId', 'homeTeamAbbr', 'visitorTeamAbbr']]


/var/folders/_g/lng396814dx5ldkl26h6w3y00000gn/T/ipykernel_85307/1281381345.py:5: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/processed/df_xPassRush.csv')


In [28]:

# left join data and games_df on gameId
df = pd.merge(data, games_df, on='gameId', how='left')
df.columns

Index(['gameId', 'playId', 'frameId', 'nflId', 'displayName', 'position',
       'club', 'possessionTeam', 'defensiveTeam', 'preSnapHomeScore',
       'preSnapVisitorScore', 'quarter', 'gameClock', 'down', 'yardsToGo',
       'yardlineNumber', 'yardlineSide', 'offenseFormation',
       'receiverAlignment', 'preSnapHomeTeamWinProbability',
       'preSnapVisitorTeamWinProbability', 'o_clean', 'a_clean', 's_clean',
       'x_clean', 'y_clean', 'dir_clean', 'playDescription',
       'passLocationType', 'rushLocationType', 'pff_runConceptPrimary',
       'yardsGained', 'wasInitialPassRusher', 'event', 'creptDist', 'center_y',
       'hDist', 'center_x', 'losDist', 'xPassRush', 'homeTeamAbbr',
       'visitorTeamAbbr'],
      dtype='object')

In [14]:
# # print number of na values in the pff_runConceptPrimary and the 'passLocationType' columns
# print(df['pff_runConceptPrimary'].unique())

# print(df['passLocationType'].unique())

# print(df['pff_runConceptPrimary'].isna().sum())
# print(df['passLocationType'].isna().sum())

# #print some rows that have na values in the 'passLocationType' column
# print(df[df['passLocationType'].isna()])

# #print total number of rows
# print(df.shape[0])

# #print number of rows where neither 'pff_runConceptPrimary' nor 'passLocationType' is na
# print(df[(df['pff_runConceptPrimary'].notna()) & (df['passLocationType'].notna())].shape[0])

# #print some rows where neither 'pff_runConceptPrimary' nor 'passLocationType' is na
# print(df[(df['pff_runConceptPrimary'].notna()) & (df['passLocationType'].notna())])

# #print unique values of pff_runConceptPrimary
# print(df['passLocationType'].value_counts())
# print(df['pff_runConceptPrimary'].value_counts())

In [29]:


# Columns representing the game context
game_context_cols = [
    "gameId",
    "playId",
    "possessionTeam",
    "defensiveTeam",
    "quarter",
    "gameClock",
    "down",
    "yardsToGo",
    "yardlineNumber",
    "yardlineSide",
    "offenseFormation",
    "receiverAlignment",
    "yardsGained",
    'preSnapHomeTeamWinProbability',
    'preSnapVisitorTeamWinProbability',
    'homeTeamAbbr',
    'visitorTeamAbbr',
    'playDescription',
    'passLocationType',
    'pff_runConceptPrimary'
]

# Filter for defensive players only
defensive_players = df[df["club"] != df["possessionTeam"]].copy()

# Add a unique identifier for each position within a play, since there can be multiple players at the same position
defensive_players["positionCount"] = (
    defensive_players.groupby(["gameId", "playId", "position"]).cumcount() + 1
)

# Create a unique column suffix for each player using their position
defensive_players["playerSuffix"] = defensive_players["position"] + defensive_players["positionCount"].astype(str)

# Pivot the defensive player hDist and losDist data
pivoted = defensive_players.pivot_table(
    index=["gameId", "playId"],
    columns="playerSuffix",
    values=["xPassRush"],
    aggfunc="first"
)

# Flatten the multi-level columns 
pivoted.columns = [f"{metric}_{suffix}" for metric, suffix in pivoted.columns]

# Reset index to merge with game context
pivoted.reset_index(inplace=True)

# Extract game context (it doesn't vary within a play)
game_context = defensive_players.drop_duplicates(subset=["gameId", "playId"])[game_context_cols]

# Merge pivoted data with game context
result_df = pd.merge(game_context, pivoted, on=["gameId", "playId"], how="left")

# compare result to original dataframe. It checks out. Since different plays will have different number of each position we generate cols for all and just use NaN for now
# print(result_df.head())
# print(result_df.columns)
# print(df[['nflId', 'playId', 'displayName','position', 'hDist', 'losDist', 'xPassRush']].head(20))
#print the counts of each unique value in the passLocationType column

#drop rows where the pff_runConceptPrimary is UNDEFINED
result_df = result_df[result_df['pff_runConceptPrimary'] != 'UNDEFINED']
#drop rows where the passLocationType is UNDEFINED
result_df = result_df[result_df['passLocationType'] != 'UNKNOWN']

# Define a function to generate the basicPlayDescription per row
def get_basic_play_description(row):
    pass_loc = row['passLocationType']
    run_concept = row['pff_runConceptPrimary']

    if pd.notna(pass_loc) and pd.notna(run_concept):
        # Both are not NaN, prioritize pass (passLocationType)
        return f"play action pass: {pass_loc}"
    elif pd.notna(pass_loc):
        # passLocationType is not NaN
        return f"pass: {pass_loc}"
    elif pd.notna(run_concept):
        # pff_runConceptPrimary is not NaN
        return f"run: {run_concept}"


# Create basicPlayDescription
result_df['basicPlayDescription'] = result_df.apply(get_basic_play_description, axis=1)

# Print the value counts of the resulting descriptions
print(result_df['basicPlayDescription'].value_counts())

#now we can drop the passLocationType and pff_runConceptPrimary columns
result_df = result_df.drop(columns=['passLocationType', 'pff_runConceptPrimary'])



basicPlayDescription
pass: INSIDE_BOX                   4998
run: OUTSIDE ZONE                  1438
play action pass: INSIDE_BOX       1248
run: INSIDE ZONE                   1128
run: MAN                            892
run: POWER                          519
run: PULL LEAD                      435
run: COUNTER                        343
pass: OUTSIDE_RIGHT                 331
play action pass: OUTSIDE_RIGHT     309
pass: OUTSIDE_LEFT                  236
run: TRICK                          182
run: DRAW                           158
play action pass: OUTSIDE_LEFT      153
run: TRAP                            90
run: FB RUN                          35
Name: count, dtype: int64


In [30]:
# Using this result, we should generate the "WinProbabilityAdded" as a target column. Recognize the PlayId is not ordered.

# Ensure the data is sorted by gameId and playId
df = result_df.sort_values(by=["gameId", "playId"]).reset_index(drop=True)

# Shift the win probability columns to get the next play's values
df["nextHomeWinProbability"] = df.groupby("gameId")["preSnapHomeTeamWinProbability"].shift(-1)

# Compute the added win probability for each play, we need only to predict this, since the addedVisitorWinProbability will just be the same value with opposite sign.
df["addedHomeWinProbability"] = df["nextHomeWinProbability"] - df["preSnapHomeTeamWinProbability"]

# Drop the temporary "next" columns
df = df.drop(columns=["nextHomeWinProbability"])

#replace NaNs with 0 in 'addedHomeWinProbability' last play of the game, actually not really sure the best way to handle these NaNs but we can't have NaNs in the target column
df['addedHomeWinProbability'] = df['addedHomeWinProbability'].fillna(0)

# Inspect the updated DataFrame
# print(df[["gameId", "playId", 'gameClock', 'down','yardsGained','yardlineNumber', "preSnapHomeTeamWinProbability", 
#           "addedHomeWinProbability"]].head(20))

# Function to calculate yards to the end zone
def calculate_yards_to_endzone(row):
    if row["yardlineSide"] == row["possessionTeam"]:
        # Driving towards the opponent's end zone
        return 100 - row["yardlineNumber"]
    else:
        # Driving towards their own end zone
        return row["yardlineNumber"]

# Apply the function to create the new column
df["yardsToEndzone"] = df.apply(calculate_yards_to_endzone, axis=1)


In [31]:
df.columns

Index(['gameId', 'playId', 'possessionTeam', 'defensiveTeam', 'quarter',
       'gameClock', 'down', 'yardsToGo', 'yardlineNumber', 'yardlineSide',
       'offenseFormation', 'receiverAlignment', 'yardsGained',
       'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability',
       'homeTeamAbbr', 'visitorTeamAbbr', 'playDescription', 'xPassRush_CB1',
       'xPassRush_CB2', 'xPassRush_CB3', 'xPassRush_CB4', 'xPassRush_CB5',
       'xPassRush_DE1', 'xPassRush_DE2', 'xPassRush_DE3', 'xPassRush_DE4',
       'xPassRush_IDL1', 'xPassRush_IDL2', 'xPassRush_IDL3', 'xPassRush_IDL4',
       'xPassRush_ILB1', 'xPassRush_ILB2', 'xPassRush_ILB3', 'xPassRush_ILB4',
       'xPassRush_OLB1', 'xPassRush_OLB2', 'xPassRush_OLB3', 'xPassRush_OLB4',
       'xPassRush_S1', 'xPassRush_S2', 'xPassRush_S3', 'xPassRush_S4',
       'xPassRush_S5', 'basicPlayDescription', 'addedHomeWinProbability',
       'yardsToEndzone'],
      dtype='object')

In [32]:
# Specify the target columns
target_yards = "yardsGained"
target_win_prob = "addedHomeWinProbability"

# specify relevant columns
cols_to_drop = ['gameId', 'yardlineSide','playId', 'possessionTeam', 'defensiveTeam', 'offenseFormation', 'receiverAlignment',
       'homeTeamAbbr', 'playDescription', 'visitorTeamAbbr', 'preSnapHomeTeamWinProbability', 'preSnapVisitorTeamWinProbability']

df_clean = df.drop(cols_to_drop, axis=1)
df_clean.columns

Index(['quarter', 'gameClock', 'down', 'yardsToGo', 'yardlineNumber',
       'yardsGained', 'xPassRush_CB1', 'xPassRush_CB2', 'xPassRush_CB3',
       'xPassRush_CB4', 'xPassRush_CB5', 'xPassRush_DE1', 'xPassRush_DE2',
       'xPassRush_DE3', 'xPassRush_DE4', 'xPassRush_IDL1', 'xPassRush_IDL2',
       'xPassRush_IDL3', 'xPassRush_IDL4', 'xPassRush_ILB1', 'xPassRush_ILB2',
       'xPassRush_ILB3', 'xPassRush_ILB4', 'xPassRush_OLB1', 'xPassRush_OLB2',
       'xPassRush_OLB3', 'xPassRush_OLB4', 'xPassRush_S1', 'xPassRush_S2',
       'xPassRush_S3', 'xPassRush_S4', 'xPassRush_S5', 'basicPlayDescription',
       'addedHomeWinProbability', 'yardsToEndzone'],
      dtype='object')

In [33]:
# Function to convert MM:SS format to float (total seconds)
def convert_game_clock_to_seconds(clock):
    minutes, seconds = map(int, clock.split(":"))
    return minutes * 60 + seconds

# Apply the conversion function to the gameClock column
df_clean['gameClock'] = df_clean['gameClock'].apply(convert_game_clock_to_seconds).astype(float)

#  orint the gameClock column
print(df_clean['gameClock'].head())


0    900.0
1    869.0
2    834.0
3    795.0
4    714.0
Name: gameClock, dtype: float64


In [34]:
print(df_clean.dtypes)

quarter                      int64
gameClock                  float64
down                         int64
yardsToGo                    int64
yardlineNumber               int64
yardsGained                  int64
xPassRush_CB1              float64
xPassRush_CB2              float64
xPassRush_CB3              float64
xPassRush_CB4              float64
xPassRush_CB5              float64
xPassRush_DE1              float64
xPassRush_DE2              float64
xPassRush_DE3              float64
xPassRush_DE4              float64
xPassRush_IDL1             float64
xPassRush_IDL2             float64
xPassRush_IDL3             float64
xPassRush_IDL4             float64
xPassRush_ILB1             float64
xPassRush_ILB2             float64
xPassRush_ILB3             float64
xPassRush_ILB4             float64
xPassRush_OLB1             float64
xPassRush_OLB2             float64
xPassRush_OLB3             float64
xPassRush_OLB4             float64
xPassRush_S1               float64
xPassRush_S2        

In [39]:
df = df_clean
# Separate categorical and numeric features
categorical_features = df.select_dtypes(include=["object", "category"]).columns
numeric_features = df.select_dtypes(include=["number"]).drop([target_yards, target_win_prob], axis=1).columns

# One-hot encode categorical features
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoded_categorical = encoder.fit_transform(df[categorical_features])
encoded_categorical_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_features))

# Combine numeric and encoded categorical features
X = pd.concat([df[numeric_features], encoded_categorical_df], axis=1)
X.columns

Index(['quarter', 'gameClock', 'down', 'yardsToGo', 'yardlineNumber',
       'xPassRush_CB1', 'xPassRush_CB2', 'xPassRush_CB3', 'xPassRush_CB4',
       'xPassRush_CB5', 'xPassRush_DE1', 'xPassRush_DE2', 'xPassRush_DE3',
       'xPassRush_DE4', 'xPassRush_IDL1', 'xPassRush_IDL2', 'xPassRush_IDL3',
       'xPassRush_IDL4', 'xPassRush_ILB1', 'xPassRush_ILB2', 'xPassRush_ILB3',
       'xPassRush_ILB4', 'xPassRush_OLB1', 'xPassRush_OLB2', 'xPassRush_OLB3',
       'xPassRush_OLB4', 'xPassRush_S1', 'xPassRush_S2', 'xPassRush_S3',
       'xPassRush_S4', 'xPassRush_S5', 'yardsToEndzone',
       'basicPlayDescription_pass: INSIDE_BOX',
       'basicPlayDescription_pass: OUTSIDE_LEFT',
       'basicPlayDescription_pass: OUTSIDE_RIGHT',
       'basicPlayDescription_play action pass: INSIDE_BOX',
       'basicPlayDescription_play action pass: OUTSIDE_LEFT',
       'basicPlayDescription_play action pass: OUTSIDE_RIGHT',
       'basicPlayDescription_run: COUNTER', 'basicPlayDescription_run: DRAW',
  

In [48]:
# Target variables
y_yards = df[target_yards]
y_win_prob = df[target_win_prob]

# Lists to store feature importances for each fold
yards_feature_importances = []
win_prob_feature_importances = []

# Define hyperparameter grid for tuning
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 1, 5],
    'reg_alpha': [0, 0.5, 1],
    'reg_lambda': [1, 2, 5]
}

# Perform KFold Cross-Validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Initialize lists to store results
yards_rmse_scores = []
win_prob_rmse_scores = []

# Perform k-fold CV
for train_index, test_index in kf.split(X):
    # Split data into training and testing sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train_yards, y_test_yards = y_yards.iloc[train_index], y_yards.iloc[test_index]
    y_train_win_prob, y_test_win_prob = y_win_prob.iloc[train_index], y_win_prob.iloc[test_index]

    # Initialize XGBoost regressor
    xgb_model = xgb.XGBRegressor(objective="reg:squarederror", random_state=42)

    # 1) Hyperparameter tuning for yards
    grid_search_yards = RandomizedSearchCV(
        estimator=xgb_model,
        param_distributions=param_grid,
        n_iter=20,  # Number of parameter settings sampled
        scoring="neg_root_mean_squared_error",
        cv=10,  # Inner cross-validation for hyperparameter tuning
        random_state=42,
        n_jobs=-1
    )
    grid_search_yards.fit(X_train, y_train_yards)

    # Best estimator for yardsGained
    best_model_yards = grid_search_yards.best_estimator_

    # 2) Hyperparameter tuning for win probability
    grid_search_win_prob = RandomizedSearchCV(
        estimator=xgb_model,
        param_distributions=param_grid,
        n_iter=20,
        scoring="neg_root_mean_squared_error",
        cv=10,
        random_state=42,
        n_jobs=-1
    )
    grid_search_win_prob.fit(X_train, y_train_win_prob)

    # Best estimator for addedHomeWinProbability
    best_model_win_prob = grid_search_win_prob.best_estimator_

    # Predict on the test set
    y_pred_yards = best_model_yards.predict(X_test)
    y_pred_win_prob = best_model_win_prob.predict(X_test)

    # Evaluate the models
    rmse_yards = np.sqrt(mean_squared_error(y_test_yards, y_pred_yards))
    rmse_win_prob = np.sqrt(mean_squared_error(y_test_win_prob, y_pred_win_prob))

    # Store the results
    yards_rmse_scores.append(rmse_yards)
    win_prob_rmse_scores.append(rmse_win_prob)

    # Store feature importance arrays for each fold
    yards_feature_importances.append(best_model_yards.feature_importances_)
    win_prob_feature_importances.append(best_model_win_prob.feature_importances_)

# Calculate and display mean and standard deviation of RMSEs
print(f"YardsGained RMSE - Mean: {np.mean(yards_rmse_scores):.4f}, Std Dev: {np.std(yards_rmse_scores):.4f}")
print(f"AddedHomeWinProbability RMSE - Mean: {np.mean(win_prob_rmse_scores):.4f}, Std Dev: {np.std(win_prob_rmse_scores):.4f}")

# Average feature importances across the 5 folds
mean_yards_importances = np.mean(yards_feature_importances, axis=0)
mean_win_prob_importances = np.mean(win_prob_feature_importances, axis=0)

# Create DataFrames for feature importances
feature_names = X.columns  # Ensure X is a DataFrame with named columns

yards_importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": mean_yards_importances
}).sort_values(by="Importance", ascending=False)

win_prob_importance_df = pd.DataFrame({
    "Feature": feature_names,
    "Importance": mean_win_prob_importances
}).sort_values(by="Importance", ascending=False)

# Print top feature importances
print("\nTop YardsGained Feature Importances:")
print(yards_importance_df.head(10))

print("\nTop WinProbability Feature Importances:")
print(win_prob_importance_df.head(10))

YardsGained RMSE - Mean: 8.5706, Std Dev: 0.2258
AddedHomeWinProbability RMSE - Mean: 0.0484, Std Dev: 0.0026

Top YardsGained Feature Importances:
                                              Feature  Importance
31                                     yardsToEndzone    0.076406
35  basicPlayDescription_play action pass: INSIDE_BOX    0.067091
32              basicPlayDescription_pass: INSIDE_BOX    0.057881
8                                       xPassRush_CB4    0.036494
43             basicPlayDescription_run: OUTSIDE ZONE    0.032667
48                          basicPlayDescription_None    0.032586
13                                      xPassRush_DE4    0.031600
24                                     xPassRush_OLB3    0.030556
20                                     xPassRush_ILB3    0.030220
33            basicPlayDescription_pass: OUTSIDE_LEFT    0.027987

Top WinProbability Feature Importances:
           Feature  Importance
2             down    0.013196
0          quarter    0

In [46]:
print(best_model_yards.get_xgb_params())

{'objective': 'reg:squarederror', 'base_score': None, 'booster': None, 'colsample_bylevel': None, 'colsample_bynode': None, 'colsample_bytree': 1.0, 'device': None, 'eval_metric': None, 'gamma': 0, 'grow_policy': None, 'interaction_constraints': None, 'learning_rate': 0.01, 'max_bin': None, 'max_cat_threshold': None, 'max_cat_to_onehot': None, 'max_delta_step': None, 'max_depth': 3, 'max_leaves': None, 'min_child_weight': None, 'monotone_constraints': None, 'multi_strategy': None, 'n_jobs': None, 'num_parallel_tree': None, 'random_state': 42, 'reg_alpha': 1, 'reg_lambda': 5, 'sampling_method': None, 'scale_pos_weight': None, 'subsample': 0.8, 'tree_method': None, 'validate_parameters': None, 'verbosity': None}
